In [1]:
from typing import *
from cps import *
from bt_search import *
from puzzleParser import *
from puzzleParser import analyze_puzzle_text


In [2]:
import pandas as pd
zebraLogicBench = pd.read_parquet("hf://datasets/allenai/ZebraLogicBench/grid_mode/test-00000-of-00001.parquet")

In [3]:
zebraLogicBench[0:2]

,id,size,puzzle,solution,created_at
0,lgp-test-5x6-16,5*6,"There are 5 houses, numbered 1 to 5 from left ...","{'header': ['House', 'Name', 'Nationality', 'B...",2024-07-11T01:22:10.734298
1,lgp-test-4x4-27,4*4,"There are 4 houses, numbered 1 to 4 from left ...","{'header': ['House', 'Name', 'Occupation', 'Bo...",2024-07-11T01:22:10.732670


In [4]:
puzzle = zebraLogicBench['puzzle'][1]
p1 = analyze_puzzle_text(puzzle)

print(p1)

Houses: 4
Vars:
 [Alice, Eric, Arnold, Peter] Each person has a unique name
 [artist, engineer, teacher, doctor] Each person has an occupation
 [fantasy, science fiction, mystery, romance] People have unique favorite book genres
 [google pixel 6, iphone 13, oneplus 9, samsung galaxy s21] People use unique phone models
Clues:
 ['engineer', 'samsung galaxy s21'] -> dLeftOf; The person who is an engineer is directly left of the person who uses a Samsung Galaxy S21
 ['fantasy'] -> is2; The person who loves fantasy books is in the second house
 ['Alice'] -> is2; Alice is not in the second house
 ['Eric', 'teacher'] -> equal; Eric is the person who is a teacher
 ['samsung galaxy s21', 'fantasy'] -> equal; The person who uses a Samsung Galaxy S21 is the person who loves fantasy books
 ['iphone 13', 'science fiction'] -> equal; The person who uses an iPhone 13 is the person who loves science fiction books
 ['science fiction', 'oneplus 9'] -> leftOf; The person who loves science fiction books i

In [5]:
def next_door(a, b):
    return a == b - 1 or a == b + 1

def right_next_to(a, b):
    return (b + 1) == a 

def direct_left_of(a, b):
    # return (a - 1) == b
    return (a + 1) == b

def left_of(a, b):
    return a < b

def right_of(a, b):
    return a > b

def one_between(a, b):
    return (a + 2) == b or (a - 2) == b

def two_between(a, b):
    return (a + 3) == b or (a - 3) == b


def configure_cps(puzzle: PzPuzzleDefinition):
    if not puzzle.is_valid():
        raise Exception("Can not generate cps for invalid puzzle definition")
    
    variables = []
    for g in puzzle.variables:
        for v in g.variables:
            variables.append(v)
    
    values = list(range(1, puzzle.house_count + 1))
    
    config = CpsConfiguration[str, int](variables, values)
    
    for g in puzzle.variables:
        config.allNotEqual(g.variables)
    
    for c in puzzle.clues:
        if c.function == "equal":
            config.equal(c.variables[0], c.variables[1])
        elif c.function == "dLeftOf":
            config.addConstraintRev(c.variables[0], c.variables[1], direct_left_of)
        elif c.function == "leftOf":
            config.addConstraintRev(c.variables[0], c.variables[1], left_of)
        elif c.function == "rightOf":
            config.addConstraintRev(c.variables[0], c.variables[1], right_of)
        elif c.function == "nextTo":
            config.addConstraintRev(c.variables[0], c.variables[1], next_door)
        elif c.function == "oneBetween":
            config.addConstraintRev(c.variables[0], c.variables[1], one_between)
        elif c.function == "twoBetween":
            config.addConstraintRev(c.variables[0], c.variables[1], two_between)
        
        elif c.function == "is1":
            config.mustBe(c.variables[0], 1)    
        elif c.function == "is2":
            config.mustBe(c.variables[0], 2)
        elif c.function == "is3":
            config.mustBe(c.variables[0], 3)    
        elif c.function == "is4":
            config.mustBe(c.variables[0], 4)
        elif c.function == "is5":
            config.mustBe(c.variables[0], 5)    
        elif c.function == "is6":
            config.mustBe(c.variables[0], 6)
        else:
            raise Exception(f'Function not implemented: "{c.function}"')
    
    return config




In [6]:
puzzle = zebraLogicBench['puzzle'][0]
p1 = analyze_puzzle_text(puzzle)
cpsConf = configure_cps(p1)
baseState = CpsState(cpsConf)
# result = BtSearch.search(SimpleBtSearch(), baseState)
result = BtSearch.search(MrvBtSearch(), baseState)

display(p1)

get_bt_result(result)

Houses: 5
Vars:
 [Peter, Alice, Bob, Eric, Arnold] Each person has a unique name
 [norwegian, german, dane, brit, swedish] The people are of nationalities
 [fantasy, biography, romance, mystery, science fiction] People have unique favorite book genres
 [stir fry, grilled cheese, pizza, spaghetti, stew] Everyone has something unique for lunch
 [red, green, blue, yellow, white] Each person has a favorite color
 [bird, dog,  cat, horse, fish] The people keep unique animals
Clues:
 ['fantasy', 'norwegian'] -> equal; The person who loves fantasy books is the Norwegian
 [' cat', 'biography'] -> nextTo; The cat lover and the person who loves biography books are next to each other
 ['german', 'Bob'] -> equal; The German is Bob
 ['yellow', 'Bob'] -> equal; The person who loves yellow is Bob
 ['green', 'Peter'] -> equal; The person whose favorite color is green is Peter
 ['dane', 'pizza'] -> oneBetween; There is one house between the Dane and the person who is a pizza lover
 ['blue', 'dane'] -> 

Step count:  46


{1: ['grilled cheese', 'Bob', 'german', 'yellow', 'mystery', 'dog'],
 2: ['norwegian', 'fantasy', 'stew', 'Eric', 'blue', 'fish'],
 3: ['Peter', 'green', 'spaghetti', 'dane', ' cat', 'science fiction'],
 4: ['Arnold', 'stir fry', 'swedish', 'red', 'bird', 'biography'],
 5: ['Alice', 'horse', 'pizza', 'brit', 'romance', 'white']}

In [8]:
failed_count = 0
failed = []

index = 0
for puzzle in zebraLogicBench['puzzle'][0:50]:
    
    p1 = analyze_puzzle_text(puzzle)
    cpsConf = configure_cps(p1)
    baseState = CpsState(cpsConf)
    # result = BtSearch.search(SimpleBtSearch(), baseState)
    result = BtSearch.search(MrvBtSearch(), baseState)
    
    if result.result is None:
        failed.append(index)
        failed_count += 1
    
    index += 1
    
display(failed_count)
display(failed)
display(failed_count / 50)

16

[2, 3, 11, 13, 16, 26, 27, 29, 32, 34, 35, 41, 42, 43, 44, 45]

0.32